__Purpose:__ Introduce Federated Learning, specifically by implementing FedAveraging on our dataset and moving on to more advanced methods.  Start by modifying the Simulations code, worry about (a)synchronicity later.
<br>
1. The dec matrix is the weights to pass back an forth (I think), although it comes out of SmoothBatch first
1. We are assuming we can test on the second half (updates 10-19ish) since learning should be complete by then!
1. Scipy.optimize.minimize() runs many iters to fully minimize its cost function.  You can change it to run as many iters as you'd like, although AFAIK you won't know how many it takes to converge.  But this is still a good set up for FL.
1. Hmm minimize() is doing BFGS rn and not SGD... not sure if that matters really.  Could probably implement SGD on my own or find it.  BFGS is 2nd order but we don't have a lot of parameters, I don't think.  Plus we can (already have?) solved analytically for the Hessian.

In [1]:
import pandas as pd
import os
import numpy as np
#from numpy.matlib import repmat
#from matplotlib import pyplot as plt
#from scipy.signal import detrend, firwin, freqz, lfilter
#from sklearn.model_selection import train_test_split, ShuffleSplit
from scipy.optimize import minimize, least_squares
import copy
from itertools import permutations

In [2]:
from experiment_params import *
from simulations import *
import time
# Do the below if you're in the pytch environment
#import pickle5 as pickle
import pickle

# Reminder of Conditions Order

NOTE: 

* **CONDITIONS** = array(['D_1', 'D_2', 'D_5', 'D_6', 'D_3', 'D_4', 'D_7','D_8']
* **LEARNING RATES:** alpha = 0.25 and 0.75; alpha = 0.25 for D1, D2, D5, D6; alpha = 0.75 for D3, D4, D7, D8
* **SMOOTHBATCH:** W_next = alpha*W_old + ((1 - alpha) * W_calc)

* **DECODER INIT:** pos for D1 - D4, neg for D5 - D8

* **PENALTY TERM:** $\lambda_E$ = 1e-6 for all, $\lambda_F$ = 1e-7 for all, $\lambda_D$ = 1e-3 for 1, 3, 5, 7 and 1e-4 for 2, 4, 6, 8 


| DECODER | ALPHA | PENALTY | DEC INIT |
| --- | --- | --- | --- |
| 1 | 0.25 | 1e-3 | + |
| 2 | 0.25 | 1e-4 | + |
| 3 | 0.75 | 1e-3 | + |
| 4 | 0.75 | 1e-4 | + |
| 5 | 0.25 | 1e-3 | - |
| 6 | 0.25 | 1e-4 | - |
| 7 | 0.75 | 1e-3 | - |
| 8 | 0.75 | 1e-4 | - |


## Load Our Data In

In [3]:
'''
t0 = time.time()
emg_data_df1 = pd.read_csv("Data\emg_full_data1.csv")
emg_data_df2 = pd.read_csv("Data\emg_full_data2.csv")
emg_data_df = pd.concat((emg_data_df1, emg_data_df2))
try:
    emg_data_df.drop('Unnamed: 0', axis=1, inplace=True)
except:
    print("NO UNNAMED COLUMN DETECTED!")
t1 = time.time()
total = t1-t0  
print(total)
print(emg_data_df.shape)
emg_data_df.head()
'''
# Just use the emg data directly from the pickle file for now
;

''

In [4]:
'''
t0 = time.time()
#envelope_df50 = pd.read_csv("Data\envelope_df50.csv")
envelope_df100 = pd.read_csv("Data\envelope_df100.csv")
#envelope_df150 = pd.read_csv("Data\envelope_df150.csv")
#envelope_df200 = pd.read_csv("Data\envelope_df200.csv")
#envelope_df250 = pd.read_csv("Data\envelope_df250.csv")
#envelope_df300 = pd.read_csv("Data\envelope_df300.csv")
#raw_envs = [envelope_df50, envelope_df100, envelope_df150, envelope_df200, envelope_df250, envelope_df300]
#all_envs = [env.drop('Unnamed: 0', axis=1) for env in raw_envs]
try:
    envelope_df100.drop('Unnamed: 0', axis=1, inplace=True)
except:
    print("NO UNNAMED COLUMN DETECTED!")
t1 = time.time()
total = t1-t0  
print(total)
print(envelope_df100.shape)
envelope_df100.head()
'''
# Just use the emg data directly from the pickle file for now
;

''

In [5]:
t0 = time.time()

with open('Data\continuous_full_data_block1.pickle', 'rb') as handle:
    #refs_block1, poss_block1, dec_vels_block1, int_vel_block1, emgs_block1, Ws_block1, Hs_block1, alphas_block1, pDs_block1, times_block1, conditions_block1 = pickle.load(handle)
    refs_block1, _, _, _, emgs_block1, Ws_block1, _, _, _, _, _ = pickle.load(handle)

#with open('Data\continuous_full_data_block2.pickle', 'rb') as handle:
    #refs_block2, poss_block2, dec_vels_block2, int_vel_block2, emgs_block2, Ws_block2, Hs_block2, alphas_block2, pDs_block2, times_block2, conditions_block2 = pickle.load(handle)
    #refs_block2, _, _, _, emgs_block2, Ws_block2, _, _, _, _, _ = pickle.load(handle)

t1 = time.time()
total = t1-t0  
print(total)

6.021999835968018


In [6]:
# 8 conditions, 20770 data points (only 19 unique sets!), xy, channels
Ws_block1[keys[0]].shape

(8, 20770, 2, 64)

In [7]:
update_ix

array([    0,  1200,  2402,  3604,  4806,  6008,  7210,  8412,  9614,
       10816, 12018, 13220, 14422, 15624, 16826, 18028, 19230, 20432,
       20769])

In [8]:
dec_cond0_user1_update0 = Ws_block1[keys[0]][0,0,:,:]
dec_cond0_user1_update1 = Ws_block1[keys[0]][0,update_ix[1],:,:]
dec_cond0_user1_update2 = Ws_block1[keys[0]][0,update_ix[2],:,:]

print(f"Shape of decoder: {dec_cond0_user1_update0.shape}")
print()
print(f"Total difference between dec0 and dec1: {(dec_cond0_user1_update0 - dec_cond0_user1_update1).sum()}")
print("E.g., as previously shown, the first two decs are the same")
print()
print(f"Total difference between dec0 and dec2: {(dec_cond0_user1_update0 - dec_cond0_user1_update2).sum()}")

Shape of decoder: (2, 64)

Total difference between dec0 and dec1: 0.0
E.g., as previously shown, the first two decs are the same

Total difference between dec0 and dec2: 3.1981579823181594


In [53]:
#emg_cond0_user1_update0 = emg_data_df.iloc[:64,:].shape

# (Condition, datapoints, channels)
print(emgs_block1[keys[0]][:,:,:].shape)

# Condition 0 of subject 1 ("0")
print(emgs_block1[keys[0]][0,:,:].shape)

(8, 20770, 64)
(20770, 64)


## Run One Iteration On Above Data and Check Decoders Are the Same
1. Modifying Simulations Code

In [54]:
# Just 1 person
filtered_signals = emgs_block1[keys[0]][0,:,:]
# Read in the reference positions from the pickle file
cued_target_position = refs_block1[keys[0]][0,:,:]

print(filtered_signals.shape)
print(cued_target_position.shape)

(20770, 64)
(20770, 2)


In [55]:
# Previously created random decoder, but we are trying to rerun
#D_0 = np.random.rand(2,64)
D_0 = Ws_block1[keys[0]][0,0,:,:]
D = []
D.append(D_0)

learning_batch = update_ix[1]

# Original simulation inputs
#alpha = .95 # higher alpha means more old decoder (slower update)
#alphaF = 1e-1
#alphaD = 1e-1
# For condition 1:
alpha = .25 # higher alpha means more old decoder (slower update)
# Assuming these are the same as lambda's, the decoder cost penalties
alphaF = 1e-7
alphaD = 1e-3
#where is lambda E?

In [58]:
#def simulation(D,learning_batch,alpha,alphaF=1e-7,alphaD=1e-3,display_info=False,num_iters=False):
display_info=True
num_iters=False

# Should I define this a priori? 
num_updates = 19

# batches the trials into each of the update batch
# Do num_updates-1 because the very last update is only 1 datapoint, the 2nd to last is only 337
for ix in range(num_updates-1):
    #print(ix)
    # For less cluttering when debugging
    #display_info = False
    
    # Instead of using learning_batch, we should get the same results just using update_ix values
    lower_bound = update_ix[ix]
    if ix==(num_updates-1):
        upper_bound = total_datapoints
    else:
        upper_bound = update_ix[ix+1]
        
    s = np.transpose(filtered_signals[lower_bound:upper_bound,:])  # Last working one
    p_intended = np.transpose(cued_target_position[lower_bound:upper_bound,:])  # This is the last working one that runs
    v_intended,p_constrained = output_new_decoder(s,D[-1],p_intended)

    # UPDATE DECODER
    u = copy.deepcopy(s) # u is the person's signal s (64 CHANNELS X TIMEPOINTS)
    q = copy.deepcopy(v_intended) # use cued positions as velocity vectors for updating decoder should be 2 x num_trials
    # emg_windows against intended_targets (trial specific cued target)
    F = copy.deepcopy(u[:,:-1]) # note: truncate F for estimate_decoder
    V = copy.deepcopy(q)

    # initial decoder estimate for gradient descent
    D0 = np.random.rand(2,64)
    # set alphas
    H = np.zeros((2,2))
    # use scipy minimize for gradient descent and provide pre-computed analytical gradient for speed
    if num_iters is False:
        out = minimize(lambda D: cost_l2(F,D,H,V,learning_batch,alphaF,alphaD), D0, method='BFGS', jac=lambda D: gradient_cost_l2(F,D,H,V,learning_batch,alphaF,alphaD), options={'disp': display_info})
    else:
        out = minimize(lambda D: cost_l2(F,D,H,V,learning_batch,alphaF,alphaD), D0, method='BFGS', jac=lambda D: gradient_cost_l2(F,D,H,V,learning_batch,alphaF,alphaD), options={'disp': display_info, 'maxiter':num_iters})

    # reshape to decoder parameters
    W_hat = np.reshape(out.x,(2, 64))

    # DO SMOOTHBATCH
    W_new = alpha*D[-1] + ((1 - alpha) * W_hat)
    D.append(W_new)

C:\Users\kdmen\AppData\Local\Temp\ipykernel_20228\1343053775.py:75: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  out = minimize(lambda D: cost_l2(F,D,H,V,learning_batch,alphaF,alphaD), D0, method='BFGS', jac=lambda D: gradient_cost_l2(F,D,H,V,learning_batch,alphaF,alphaD), options={'disp': display_info})


Optimization terminated successfully.
         Current function value: 137.753028
         Iterations: 92
         Function evaluations: 126
         Gradient evaluations: 126
Optimization terminated successfully.
         Current function value: 126.003342
         Iterations: 102
         Function evaluations: 134
         Gradient evaluations: 134
Optimization terminated successfully.
         Current function value: 170.284743
         Iterations: 94
         Function evaluations: 123
         Gradient evaluations: 123
Optimization terminated successfully.
         Current function value: 178.425631
         Iterations: 93
         Function evaluations: 126
         Gradient evaluations: 126
Optimization terminated successfully.
         Current function value: 167.921070
         Iterations: 93
         Function evaluations: 123
         Gradient evaluations: 123
Optimization terminated successfully.
         Current function value: 124.951363
         Iterations: 92
         Func

p_int: (2, 1202) <br>
v_int: (2, 1202) <br>
u: (64, 1202) <br>
q: (2, 1202) <br>
F: (64, 1201) <br>
V: (2, 1202) <br>

In [ ]:
# given decoder, what is the new position?
def output_new_decoder(s,D,p_intended):
    '''
    s: (64 x (60 timepoints x learning batch size))
    D: (2 x 64) previous D computed or random
    p_intended: (2 x 60 timepoints x learning batch size)
    '''
    # take first trial, random decoder, and do target classification
    v = D@s # actual decoded velocity (2,60)

    # integrate decoded velocities into positions
    p = []
    for ix in range(v.shape[1]):
        p.append(np.sum(v[:,:ix],axis=1))
    p = np.asarray(p).T # actual decoded position (2,60)

    # want error between intended and actual velocity but need to constrain actual velocity to target radius
    p_constrained = np.asarray([constrain_p_actual(p_) for p_ in p.T]).T # constrained, (2,60)
    # compute error between intended and actual position and take derivative to get intended velocity
    v_intended = p_intended - p_constrained # (2,60)
    return v_intended,p_constrained

In [ ]:
#import os
#import copy
#import time
#import pickle
#import numpy as np
#from tqdm import tqdm

#import torch
#from tensorboardX import SummaryWriter

#from ARJ_options import args_parser
#from ARJ_update import LocalUpdate, test_inference
#from ARJ_my_models import MLP, CNNMnist, CNNFashion_Mnist, CNNCifar
#from ARJ_utils import get_dataset, average_weights, exp_details


if __name__ == '__main__':
    #start_time = time.time()

    # define paths
    #path_project = os.path.abspath('..')
    #logger = SummaryWriter('../logs')
    
    # This is actually useful.  Real SWE stuff.
    args = args_parser()
    exp_details(args)

    #if args.gpu_id:
    #    torch.cuda.set_device(args.gpu_id)
    #device = 'cuda' if args.gpu else 'cpu'

    # load dataset and user groups
    train_dataset, test_dataset, user_groups = get_dataset(args)

    # BUILD MODEL
    #if args.model == 'cnn':
    #    # Convolutional neural netork
    #    if args.dataset == 'mnist':
    #        global_model = CNNMnist(args=args)
    #    elif args.dataset == 'fmnist':
    #        global_model = CNNFashion_Mnist(args=args)
    #    elif args.dataset == 'cifar':
    #        global_model = CNNCifar(args=args)
    #elif args.model == 'mlp':
    #    # Multi-layer preceptron
    #    img_size = train_dataset[0][0].shape
    #    len_in = 1
    #    for x in img_size:
    #        len_in *= x
    #        global_model = MLP(dim_in=len_in, dim_hidden=64,
    #                           dim_out=args.num_classes)
    #else:
    #    exit('Error: unrecognized model')
    # So for us... just set it to linear regression I guess lol
    '''global_model = linear_regression (D@s)'''

    # Set the model to train and send it to device.
    # KAI: I think this actually sends it to the GPU, not a "client" in the FL sense
    #global_model.to(device)
    global_model.train()
    print(global_model)

    # copy weights
    global_weights = global_model.state_dict()

    # Training
    train_loss, train_accuracy = [], []
    val_acc_list, net_list = [], []
    cv_loss, cv_acc = [], []
    print_every = 2
    val_loss_pre, counter = 0, 0

    for epoch in tqdm(range(args.epochs)):
        local_weights, local_losses = [], []
        print(f'\n | Global Training Round : {epoch+1} |\n')

        global_model.train()
        m = max(int(args.frac * args.num_users), 1)
        idxs_users = np.random.choice(range(args.num_users), m, replace=False)

        for idx in idxs_users:
            local_model = LocalUpdate(args=args, dataset=train_dataset,
                                      idxs=user_groups[idx], logger=logger)
            w, loss = local_model.update_weights(
                model=copy.deepcopy(global_model), global_round=epoch)
            local_weights.append(copy.deepcopy(w))
            local_losses.append(copy.deepcopy(loss))

        # update global weights
        global_weights = average_weights(local_weights)

        # update global weights
        global_model.load_state_dict(global_weights)

        loss_avg = sum(local_losses) / len(local_losses)
        train_loss.append(loss_avg)

        # Calculate avg training accuracy over all users at every epoch
        list_acc, list_loss = [], []
        global_model.eval()
        for c in range(args.num_users):
            local_model = LocalUpdate(args=args, dataset=train_dataset,
                                      idxs=user_groups[idx], logger=logger)
            acc, loss = local_model.inference(model=global_model)
            list_acc.append(acc)
            list_loss.append(loss)
        train_accuracy.append(sum(list_acc)/len(list_acc))

        # print global training loss after every 'i' rounds
        if (epoch+1) % print_every == 0:
            print(f' \nAvg Training Stats after {epoch+1} global rounds:')
            print(f'Training Loss : {np.mean(np.array(train_loss))}')
            print('Train Accuracy: {:.2f}% \n'.format(100*train_accuracy[-1]))

    # Test inference after completion of training
    test_acc, test_loss = test_inference(args, global_model, test_dataset)

    print(f' \n Results after {args.epochs} global rounds of training:')
    print("|---- Avg Train Accuracy: {:.2f}%".format(100*train_accuracy[-1]))
    print("|---- Test Accuracy: {:.2f}%".format(100*test_acc))

    # Saving the objects train_loss and train_accuracy:
    file_name = '../save/objects/{}_{}_{}_C[{}]_iid[{}]_E[{}]_B[{}].pkl'.\
        format(args.dataset, args.model, args.epochs, args.frac, args.iid,
               args.local_ep, args.local_bs)

    with open(file_name, 'wb') as f:
        pickle.dump([train_loss, train_accuracy], f)

    print('\n Total Run Time: {0:0.4f}'.format(time.time()-start_time))

    # PLOTTING (optional)
    # import matplotlib
    # import matplotlib.pyplot as plt
    # matplotlib.use('Agg')

    # Plot Loss curve
    # plt.figure()
    # plt.title('Training Loss vs Communication rounds')
    # plt.plot(range(len(train_loss)), train_loss, color='r')
    # plt.ylabel('Training loss')
    # plt.xlabel('Communication Rounds')
    # plt.savefig('../save/fed_{}_{}_{}_C[{}]_iid[{}]_E[{}]_B[{}]_loss.png'.
    #             format(args.dataset, args.model, args.epochs, args.frac,
    #                    args.iid, args.local_ep, args.local_bs))
    #
    # # Plot Average Accuracy vs Communication rounds
    # plt.figure()
    # plt.title('Average Accuracy vs Communication rounds')
    # plt.plot(range(len(train_accuracy)), train_accuracy, color='k')
    # plt.ylabel('Average Accuracy')
    # plt.xlabel('Communication Rounds')
    # plt.savefig('../save/fed_{}_{}_{}_C[{}]_iid[{}]_E[{}]_B[{}]_acc.png'.
    #             format(args.dataset, args.model, args.epochs, args.frac,
    #                    args.iid, args.local_ep, args.local_bs))

In [ ]:
#import os
#import copy
#import time
#import pickle
#import numpy as np
#from tqdm import tqdm

#import torch
#from tensorboardX import SummaryWriter

#from ARJ_options import args_parser
#from ARJ_update import LocalUpdate, test_inference
#from ARJ_my_models import MLP, CNNMnist, CNNFashion_Mnist, CNNCifar
#from ARJ_utils import get_dataset, average_weights, exp_details



# define paths
#path_project = os.path.abspath('..')
#logger = SummaryWriter('../logs')

# This is actually useful.  Real SWE stuff.
args = args_parser()
exp_details(args)

# load dataset and user groups
train_dataset, test_dataset, user_groups = get_dataset(args)

# BUILD MODEL
# So for us... just set it to linear regression I guess lol
# I don't know if I need to explicitly define a model object... would make things more legible tho
'''global_model = linear_regression (D@s)'''
class model():
    pass

# Set the model to train and send it to device.
# KAI: I think this actually sends it to the GPU, not a "client" in the FL sense
#global_model.to(device)
global_model.train()
print(global_model)
# minimize()?

# copy weights
#global_weights = global_model.state_dict()
'''global_weights = D'''

# Training
train_loss, train_accuracy = [], []
val_acc_list, net_list = [], []
cv_loss, cv_acc = [], []
print_every = 2
val_loss_pre, counter = 0, 0

for epoch in tqdm(range(args.epochs)):
    local_weights, local_losses = [], []
    print(f'\n | Global Training Round : {epoch+1} |\n')

    global_model.train()
    m = max(int(args.frac * args.num_users), 1)
    idxs_users = np.random.choice(range(args.num_users), m, replace=False)

    for idx in idxs_users:
        local_model = LocalUpdate(args=args, dataset=train_dataset,
                                  idxs=user_groups[idx], logger=logger)
        w, loss = local_model.update_weights(
            model=copy.deepcopy(global_model), global_round=epoch)
        local_weights.append(copy.deepcopy(w))
        local_losses.append(copy.deepcopy(loss))

    # update global weights
    global_weights = average_weights(local_weights)

    # update global weights
    global_model.load_state_dict(global_weights)

    loss_avg = sum(local_losses) / len(local_losses)
    train_loss.append(loss_avg)

    # Calculate avg training accuracy over all users at every epoch
    list_acc, list_loss = [], []
    global_model.eval()
    for c in range(args.num_users):
        local_model = LocalUpdate(args=args, dataset=train_dataset,
                                  idxs=user_groups[idx], logger=logger)
        acc, loss = local_model.inference(model=global_model)
        list_acc.append(acc)
        list_loss.append(loss)
    train_accuracy.append(sum(list_acc)/len(list_acc))

    # print global training loss after every 'i' rounds
    if (epoch+1) % print_every == 0:
        print(f' \nAvg Training Stats after {epoch+1} global rounds:')
        print(f'Training Loss : {np.mean(np.array(train_loss))}')
        print('Train Accuracy: {:.2f}% \n'.format(100*train_accuracy[-1]))

# Test inference after completion of training
test_acc, test_loss = test_inference(args, global_model, test_dataset)

print(f' \n Results after {args.epochs} global rounds of training:')
print("|---- Avg Train Accuracy: {:.2f}%".format(100*train_accuracy[-1]))
print("|---- Test Accuracy: {:.2f}%".format(100*test_acc))

# Saving the objects train_loss and train_accuracy:
file_name = '../save/objects/{}_{}_{}_C[{}]_iid[{}]_E[{}]_B[{}].pkl'.\
    format(args.dataset, args.model, args.epochs, args.frac, args.iid,
           args.local_ep, args.local_bs)

with open(file_name, 'wb') as f:
    pickle.dump([train_loss, train_accuracy], f)

print('\n Total Run Time: {0:0.4f}'.format(time.time()-start_time))

# PLOTTING (optional)
# import matplotlib
# import matplotlib.pyplot as plt
# matplotlib.use('Agg')

# Plot Loss curve
# plt.figure()
# plt.title('Training Loss vs Communication rounds')
# plt.plot(range(len(train_loss)), train_loss, color='r')
# plt.ylabel('Training loss')
# plt.xlabel('Communication Rounds')
# plt.savefig('../save/fed_{}_{}_{}_C[{}]_iid[{}]_E[{}]_B[{}]_loss.png'.
#             format(args.dataset, args.model, args.epochs, args.frac,
#                    args.iid, args.local_ep, args.local_bs))
#
# # Plot Average Accuracy vs Communication rounds
# plt.figure()
# plt.title('Average Accuracy vs Communication rounds')
# plt.plot(range(len(train_accuracy)), train_accuracy, color='k')
# plt.ylabel('Average Accuracy')
# plt.xlabel('Communication Rounds')
# plt.savefig('../save/fed_{}_{}_{}_C[{}]_iid[{}]_E[{}]_B[{}]_acc.png'.
#             format(args.dataset, args.model, args.epochs, args.frac,
#                    args.iid, args.local_ep, args.local_bs))